In [2]:
import time
from datetime import datetime
import numpy as np
import torch
import torchvision
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.animation import FuncAnimation
from conv_model import ConvLayer
from small_data_classifier import SmallDataClassifier

from sklearn.model_selection import train_test_split

from train_conv3d_model import plot_original_vs_recon
from train_conv3d_model import plot_filters_2d, plot_filters_3d
from train_conv3d_model import plot_video

from load_data import load_bamc_data, load_covid_data

from IPython.display import HTML

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device == "cpu":
    batch_size = 1
else:
    batch_size = 4*3
    # batch_size = 3

# train_loader = load_balls_data(batch_size)
train_loader, test_loader = load_mnist_data(batch_size)

example_data = next(iter(train_loader))

layer = ConvLayer(in_channels=1,
                               out_channels=24,
                               kernel_size=(16, 16),
                               stride=2,
                               padding=0,
                               convo_dim=2)
model = layer
model = torch.nn.DataParallel(model, device_ids=[0, 1, 2, 3])
model.to(device)

In [ ]:
# Load models if we'd like to
checkpoint = torch.load("conv_base/sparse_conv3d_model-best.pt")
model.module.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
ani = plot_video(example_data[1][1])
HTML(ani.to_html5_video())

In [ ]:
idx=1
reconstructions = layer(
    example_data[1][idx:idx+1].to(device)).cpu().detach().numpy()

ani = plot_video(reconstructions.squeeze(0))
# ani = plot_original_vs_recon(example_data[1][idx:idx+1], reconstructions, idx=0)
HTML(ani.to_html5_video())

In [ ]:
print(layer.convo.weight.data.cpu().detach().shape)
ani = plot_filters(layer.convo.weight.data.cpu().detach())
HTML(ani.to_html5_video())
# ani.save('conv_base/frame_vis.mp4')